# API

In [8]:
# Import the requests library

import requests
import pandas as pd

# Call the API and check that everything is OK. If OK, status_code will print out 200.
iss_now_response = requests.get("http://api.open-notify.org/iss-now.json")

print(iss_now_response.status_code)

200


In [2]:
# Convert data into a Python object
location_data = iss_now_response.json()

# Convert "timestamp" from UNIX timestamp to readable format
import datetime
location_timestamp = datetime.datetime.fromtimestamp(
        location_data["timestamp"]
    ).strftime('%Y-%m-%d %H:%M:%S')

# Print the most current ISS position in latitude/longitude and the time that it was there.
print("The ISS position at", location_timestamp, "is", location_data["iss_position"])

# Pull out the latitude and longitude to be used in the next step
iss_lat = location_data["iss_position"]["latitude"]
iss_lon = location_data["iss_position"]["longitude"]

iss_lat = float(iss_lat)
iss_lon = float(iss_lon)

print("Latitude:", iss_lat)
print("Longitude:", iss_lon)

The ISS position at 2020-02-25 16:48:01 is {'longitude': '-163.3041', 'latitude': '40.9568'}
Latitude: 40.9568
Longitude: -163.3041


In [3]:
import reverse_geocoder as rg

coordinates = (iss_lat, iss_lon)

results = rg.search(coordinates) # default mode = 2

print(results)
#print(f'The location of the ISS at this moment is {results[0][0]}.')

for x in results:
  print(f' The location of the ISS at this moment is {x["admin1"]}')

    

Loading formatted geocoded file...
[OrderedDict([('lat', '53.87361'), ('lon', '-166.53667'), ('name', 'Unalaska'), ('admin1', 'Alaska'), ('admin2', 'Aleutians West Census Area'), ('cc', 'US')])]
 The location of the ISS at this moment is Alaska


In [4]:
 #Set up the parameters as a dictionary, which we'll then pass to the API
# This is the latitude, longitude, and altitude of St. Louis, Missouri.

#parameters = {"lat": 38.627003, "lon": -90.199402, "alt":466, "n":10}

# Make a get request with the parameters.
#stl_response = requests.get("http://api.open-notify.org/iss-pass.json", params=parameters)

# Print the content of the response (the data the server returned)
#print(stl_response.content)

# Alternative approach: this gets the same data as the command above
# .decode("utf-8") converts it to string format

#duration in seconds

stl2_response = requests.get("http://api.open-notify.org/iss-pass.json?lat=41.3851&lon=2.1734&n=10")
print("\n", stl2_response.content.decode("utf-8"))


 {
  "message": "success", 
  "request": {
    "altitude": 100, 
    "datetime": 1582645684, 
    "latitude": 41.3851, 
    "longitude": 2.1734, 
    "passes": 10
  }, 
  "response": [
    {
      "duration": 402, 
      "risetime": 1582684252
    }, 
    {
      "duration": 647, 
      "risetime": 1582689895
    }, 
    {
      "duration": 628, 
      "risetime": 1582695721
    }, 
    {
      "duration": 576, 
      "risetime": 1582701605
    }, 
    {
      "duration": 613, 
      "risetime": 1582707451
    }, 
    {
      "duration": 657, 
      "risetime": 1582713258
    }, 
    {
      "duration": 517, 
      "risetime": 1582719105
    }, 
    {
      "duration": 152, 
      "risetime": 1582767937
    }, 
    {
      "duration": 624, 
      "risetime": 1582773454
    }, 
    {
      "duration": 642, 
      "risetime": 1582779248
    }
  ]
}



In [5]:
from datetime import datetime

timestamp = 1545730073
dt_object = datetime.fromtimestamp(timestamp)

print("The time is:", dt_object)
#print("type(dt_object) =", type(dt_object))

The time is: 2018-12-25 10:27:53


In [6]:
# Get the response from the API endpoint.
response = requests.get("http://api.open-notify.org/astros.json")
astros_data = response.json()

# How many people are currently in space?
print("Number of people currently in space: ", astros_data["number"])
print("\nData:", astros_data)

Number of people currently in space:  3

Data: {'people': [{'craft': 'ISS', 'name': 'Andrew Morgan'}, {'craft': 'ISS', 'name': 'Oleg Skripochka'}, {'craft': 'ISS', 'name': 'Jessica Meir'}], 'message': 'success', 'number': 3}


In [10]:
#pd.DataFrame.from_dict(astros_data)

df = pd.DataFrame()
df['info'] = astros_data['people']
#df_beer_styles['style'] = style_list

df.head()

,info
0,"{'craft': 'ISS', 'name': 'Andrew Morgan'}"
1,"{'craft': 'ISS', 'name': 'Oleg Skripochka'}"
2,"{'craft': 'ISS', 'name': 'Jessica Meir'}"


# Scraping 

In [15]:
import json
import requests
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup


In [23]:
def getMissionInfo():
    html = requests.get("https://spaceflightnow.com/launch-schedule/").text
    soup = BeautifulSoup(html, 'html.parser')
    #print(type(soup))
    missionList = []
    counter = 0
    
    # launchList contains the flight window and launch location
    launchList = []
    for i in soup.find_all('div', attrs={"class": 'missiondata'}):
        launchTime = i.text
        launchList.append(launchTime)

    # descriptionList contains the mission descriptions
    descriptionList = []
    for i in soup.find_all('div', attrs={"class": 'missdescrip'}):
        missiondescription = i.text
        descriptionList.append(missiondescription)

    # the date and mission name are included before the entire row is appended to the missionList
    for i in soup.find_all('div', attrs={'class': 'datename'}):
        date = i.find('span', attrs={'class': 'launchdate'})
        mission = i.find('span', attrs={'class': 'mission'})
        row = {
                "date": date.text, "name":mission.text,
                "launchInfo": launchList[int(counter)],
                "description":descriptionList[int(counter)]
            }
        missionList.append(row)
        counter += 1
    return missionList

In [24]:
#getMissionInfo()

In [25]:
missions = pd.DataFrame.from_dict(getMissionInfo())
missions.head()

,date,name,launchInfo,description
0,NET Feb. 27,Astra Rocket 3.0 • DARPA Launch Challenge 1,Launch window: 2030-2330 GMT (3:30-6:30 p.m. E...,Astra Space’s Rocket 3.0 will lift off on its ...
1,March 5,GSLV Mk.2 • GISAT 1,Launch time: TBD\nLaunch site: Satish Dhawan S...,India’s Geosynchronous Satellite Launch Vehicl...
2,TBD,Falcon 9 • SpaceX CRS 20,"Launch time: TBD\nLaunch site: SLC-40, Cape Ca...",A SpaceX Falcon 9 rocket will launch the 22nd ...
3,March 6,Soyuz • Falcon Eye 2,Launch time: 0133 GMT on 6th (8:33 p.m. EST)\n...,"An Arianespace Soyuz rocket, designated VS24, ..."
4,March 11,Falcon 9 • Starlink 5,Launch time: Approx. 1440 GMT (10:40 a.m. EDT)...,A SpaceX Falcon 9 rocket is expected to launch...
